In [2]:
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


c:\Users\73bib\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# =====================
# 1. 데이터 불러오기
# =====================

csv_path = r"C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\project\YT_ChannelGrowth_Engagement\data\processed\youtube_channels_clean_v2.csv"
df = pd.read_csv(csv_path, low_memory=False)

print("채널 데이터 로드 완료:", df.shape)
print("컬럼 목록:", list(df.columns))


채널 데이터 로드 완료: (15830, 17)
컬럼 목록: ['channel_id', 'channel_name', 'subscriber_count', 'view_count', 'video_count', 'created_date', 'category', 'country', 'videos_last_30_days', 'views_last_30_days', 'channel_age_days', 'upload_frequency', 'subscriber_per_view', 'views_per_video', 'uploads_per_subscriber', 'category_encoded', 'country_encoded']


In [4]:
# =====================
# 2. 입력 변수 / 타깃 설정
#    (3번에서 쓰던 것과 동일하게 맞춤)
# =====================

feature_cols = [
    'upload_frequency',
    'views_per_video',
    'subscriber_per_view',
    'video_count',
    'channel_age_days',
    'category_encoded',
    'country_encoded',
]

target_col = 'views_last_30_days'

# 방어 코드: 컬럼 존재 체크
missing_feats = [c for c in feature_cols if c not in df.columns]
if missing_feats:
    raise ValueError(f"아래 feature 컬럼이 csv에 없습니다: {missing_feats}")

if target_col not in df.columns:
    raise ValueError(f"타깃 컬럼 '{target_col}' 이(가) csv에 없습니다!")

X = df[feature_cols].copy()
y = df[target_col].copy()

# 결측치 간단 처리
X = X.fillna(0)
y = y.fillna(0)

print("\n사용 독립변수(feature_cols):", feature_cols)
print("타깃 변수:", target_col)



사용 독립변수(feature_cols): ['upload_frequency', 'views_per_video', 'subscriber_per_view', 'video_count', 'channel_age_days', 'category_encoded', 'country_encoded']
타깃 변수: views_last_30_days


In [5]:
# =====================
# 3. Train / Test 분할
# =====================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("\nTrain size:", X_train.shape, " / Test size:", X_test.shape)



Train size: (12664, 7)  / Test size: (3166, 7)


In [6]:
# =====================
# 4. RandomForest 회귀 모델 학습
# =====================

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=20,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
print("RandomForest 회귀 학습 완료")


RandomForest 회귀 학습 완료


In [7]:
# =====================
# 5. SHAP 값 계산 (채널 성장)
# =====================

# 샘플링 (채널 15830개라 전부 써도 되지만, 안전하게 3000개 제한)
if len(X_train) > 3000:
    X_sample = X_train.sample(3000, random_state=42)
else:
    X_sample = X_train.copy()

print("SHAP 계산용 샘플 크기:", X_sample.shape)

explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_sample)

print("SHAP 값 계산 완료")


SHAP 계산용 샘플 크기: (3000, 7)
SHAP 값 계산 완료


In [13]:
# =====================
# 6. SHAP 시각화 저장
# =====================

out_dir = r"C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\project\YT_ChannelGrowth_Engagement\reports\figures\shap"

# 6-1. 변수 중요도 막대그래프 (mean(|SHAP|))
plt.figure()
shap.summary_plot(
    shap_values,
    X_sample,
    plot_type="bar",
    show=False
)

bar_path = out_dir + r"\shap_channel_growth_bar.png"
plt.tight_layout()
plt.savefig(bar_path, dpi=200, bbox_inches="tight")
plt.close()
print(f"채널 성장 SHAP Bar Plot 저장 완료 → {bar_path}")

# 6-2. summary dot plot (특징별 양/음 영향까지)
plt.figure()
shap.summary_plot(
    shap_values,
    X_sample,
    show=False
)
dot_path = out_dir + r"\shap_channel_growth_summary.png"
plt.tight_layout()
plt.savefig(dot_path, dpi=200, bbox_inches="tight")
plt.close()
print(f"채널 성장 SHAP Summary Plot 저장 완료 → {dot_path}")

print("\n채널 성장 SHAP 분석 완료! (회귀 모델 기준)")


채널 성장 SHAP Bar Plot 저장 완료 → C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\project\YT_ChannelGrowth_Engagement\reports\figures\shap\shap_channel_growth_bar.png
채널 성장 SHAP Summary Plot 저장 완료 → C:\Users\73bib\Desktop\유혜원\제주한라대학교\[2025] 1학년 2학기\빅데이터 기초 및 실습\project\YT_ChannelGrowth_Engagement\reports\figures\shap\shap_channel_growth_summary.png

채널 성장 SHAP 분석 완료! (회귀 모델 기준)
